### Book Recommander System using clustering | Collaborative filtering based

In [1]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns
import warnings

In [2]:
warnings.filterwarnings('ignore')

In [3]:
books = pd.read_csv('data/books.csv', sep=";", error_bad_lines=False, encoding='latin-1')

b'Skipping line 6452: expected 8 fields, saw 9\nSkipping line 43667: expected 8 fields, saw 10\nSkipping line 51751: expected 8 fields, saw 9\n'
b'Skipping line 92038: expected 8 fields, saw 9\nSkipping line 104319: expected 8 fields, saw 9\nSkipping line 121768: expected 8 fields, saw 9\n'
b'Skipping line 144058: expected 8 fields, saw 9\nSkipping line 150789: expected 8 fields, saw 9\nSkipping line 157128: expected 8 fields, saw 9\nSkipping line 180189: expected 8 fields, saw 9\nSkipping line 185738: expected 8 fields, saw 9\n'
b'Skipping line 209388: expected 8 fields, saw 9\nSkipping line 220626: expected 8 fields, saw 9\nSkipping line 227933: expected 8 fields, saw 11\nSkipping line 228957: expected 8 fields, saw 10\nSkipping line 245933: expected 8 fields, saw 9\nSkipping line 251296: expected 8 fields, saw 9\nSkipping line 259941: expected 8 fields, saw 9\nSkipping line 261529: expected 8 fields, saw 9\n'


In [4]:
books.head()

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...


In [5]:
books.shape

(271360, 8)

In [6]:
books.columns

Index(['ISBN', 'Book-Title', 'Book-Author', 'Year-Of-Publication', 'Publisher',
       'Image-URL-S', 'Image-URL-M', 'Image-URL-L'],
      dtype='object')

In [7]:
books.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 271360 entries, 0 to 271359
Data columns (total 8 columns):
 #   Column               Non-Null Count   Dtype 
---  ------               --------------   ----- 
 0   ISBN                 271360 non-null  object
 1   Book-Title           271360 non-null  object
 2   Book-Author          271359 non-null  object
 3   Year-Of-Publication  271360 non-null  object
 4   Publisher            271358 non-null  object
 5   Image-URL-S          271360 non-null  object
 6   Image-URL-M          271360 non-null  object
 7   Image-URL-L          271357 non-null  object
dtypes: object(8)
memory usage: 16.6+ MB


In [8]:
books = books[['ISBN', 'Book-Title', 'Book-Author', 'Year-Of-Publication', 'Publisher','Image-URL-L']]

In [9]:
books.head(2)

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-L
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...


In [10]:
books.rename(columns={
    "Book-Title" : "Title",
    "Book-Author": "Author",
    "Year-Of-Publication": "Year",
    "Image-URL-L": "Image_url"}, inplace=True)

In [11]:
books.head(2)

,ISBN,Title,Author,Year,Publisher,Image_url
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...


In [12]:
users = pd.read_csv('data/users.csv', sep=";", error_bad_lines=False, encoding='latin-1')

In [13]:
users.head(2)

,User-ID,Location,Age
0,1,"nyc, new york, usa",NaN
1,2,"stockton, california, usa",18.0


In [14]:
users.shape

(278858, 3)

In [15]:
users.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 278858 entries, 0 to 278857
Data columns (total 3 columns):
 #   Column    Non-Null Count   Dtype  
---  ------    --------------   -----  
 0   User-ID   278858 non-null  int64  
 1   Location  278858 non-null  object 
 2   Age       168096 non-null  float64
dtypes: float64(1), int64(1), object(1)
memory usage: 6.4+ MB


In [16]:
users.describe()

,User-ID,Age
count,278858.00000,168096.000000
mean,139429.50000,34.751434
std,80499.51502,14.428097
min,1.00000,0.000000
25%,69715.25000,24.000000
50%,139429.50000,32.000000
75%,209143.75000,44.000000
max,278858.00000,244.000000


In [17]:
ratings = pd.read_csv('data/ratings.csv', sep=";", error_bad_lines=False, encoding='latin-1');

In [18]:
ratings.shape

(1149780, 3)

In [19]:
ratings.head(3)

,User-ID,ISBN,Book-Rating
0,276725,034545104X,0
1,276726,0155061224,5
2,276727,0446520802,0


In [20]:
ratings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1149780 entries, 0 to 1149779
Data columns (total 3 columns):
 #   Column       Non-Null Count    Dtype 
---  ------       --------------    ----- 
 0   User-ID      1149780 non-null  int64 
 1   ISBN         1149780 non-null  object
 2   Book-Rating  1149780 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 26.3+ MB


### Missing information that needs to be handled

In [21]:
print(books.shape)
print(users.shape)
print(ratings.shape)

(271360, 6)
(278858, 3)
(1149780, 3)


In [22]:
ratings['User-ID'].value_counts()

11676     13602
198711     7550
153662     6109
98391      5891
35859      5850
          ...  
116180        1
116166        1
116154        1
116137        1
276723        1
Name: User-ID, Length: 105283, dtype: int64

In [23]:
ratings['User-ID'].unique().shape

(105283,)

In [24]:
x = ratings['User-ID'].value_counts() > 200 

In [25]:
x

11676      True
198711     True
153662     True
98391      True
35859      True
          ...  
116180    False
116166    False
116154    False
116137    False
276723    False
Name: User-ID, Length: 105283, dtype: bool

In [26]:
x[x].shape

(899,)

In [27]:
y = x[x].index

In [28]:
y

Int64Index([ 11676, 198711, 153662,  98391,  35859, 212898, 278418,  76352,
            110973, 235105,
            ...
            260183,  73681,  44296, 155916,   9856, 274808,  28634,  59727,
            268622, 188951],
           dtype='int64', length=899)

In [29]:
ratings = ratings[ratings['User-ID'].isin(y)]

In [30]:
ratings

,User-ID,ISBN,Book-Rating
1456,277427,002542730X,10
1457,277427,0026217457,0
1458,277427,003008685X,8
1459,277427,0030615321,0
1460,277427,0060002050,0
...,...,...,...
1147612,275970,3829021860,0
1147613,275970,4770019572,0
1147614,275970,896086097,0
1147615,275970,9626340762,8


In [31]:
ratings.head()

,User-ID,ISBN,Book-Rating
1456,277427,002542730X,10
1457,277427,0026217457,0
1458,277427,003008685X,8
1459,277427,0030615321,0
1460,277427,0060002050,0


In [32]:
books.head()

,ISBN,Title,Author,Year,Publisher,Image_url
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,http://images.amazon.com/images/P/0060973129.0...
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux,http://images.amazon.com/images/P/0374157065.0...
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company,http://images.amazon.com/images/P/0393045218.0...


In [33]:
ratings_with_books = ratings.merge(books, on = 'ISBN')

In [34]:
ratings_with_books

,User-ID,ISBN,Book-Rating,Title,Author,Year,Publisher,Image_url
0,277427,002542730X,10,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,http://images.amazon.com/images/P/002542730X.0...
1,3363,002542730X,0,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,http://images.amazon.com/images/P/002542730X.0...
2,11676,002542730X,6,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,http://images.amazon.com/images/P/002542730X.0...
3,12538,002542730X,10,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,http://images.amazon.com/images/P/002542730X.0...
4,13552,002542730X,0,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,http://images.amazon.com/images/P/002542730X.0...
...,...,...,...,...,...,...,...,...
487666,275970,1892145022,0,Here Is New York,E. B. White,1999,Little Bookroom,http://images.amazon.com/images/P/1892145022.0...
487667,275970,1931868123,0,There's a Porcupine in My Outhouse: Misadventu...,Mike Tougias,2002,Capital Books (VA),http://images.amazon.com/images/P/1931868123.0...
487668,275970,3411086211,10,Die Biene.,Sybil GrÃ?Â¤fin SchÃ?Â¶nfeldt,1993,"Bibliographisches Institut, Mannheim",http://images.amazon.com/images/P/3411086211.0...
487669,275970,3829021860,0,The Penis Book,Joseph Cohen,1999,Konemann,http://images.amazon.com/images/P/3829021860.0...


In [35]:
ratings_with_books.shape

(487671, 8)

In [36]:
num_rating = ratings_with_books.groupby('Title')['Book-Rating'].count().reset_index()

In [37]:
num_rating.head()

,Title,Book-Rating
0,A Light in the Storm: The Civil War Diary of ...,2
1,Always Have Popsicles,1
2,Apple Magic (The Collector's series),1
3,Beyond IBM: Leadership Marketing and Finance ...,1
4,Clifford Visita El Hospital (Clifford El Gran...,1


In [38]:
num_rating.rename(columns={'Book-Rating': 'Num-Of-Rating'}, inplace= True)

In [39]:
ratings_with_books.head(2)

,User-ID,ISBN,Book-Rating,Title,Author,Year,Publisher,Image_url
0,277427,002542730X,10,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,http://images.amazon.com/images/P/002542730X.0...
1,3363,002542730X,0,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,http://images.amazon.com/images/P/002542730X.0...


In [40]:
final_rating = ratings_with_books.merge(num_rating, on = 'Title')

In [41]:
final_rating.head(2)

,User-ID,ISBN,Book-Rating,Title,Author,Year,Publisher,Image_url,Num-Of-Rating
0,277427,002542730X,10,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,http://images.amazon.com/images/P/002542730X.0...,82
1,3363,002542730X,0,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,http://images.amazon.com/images/P/002542730X.0...,82


In [42]:
final_rating = final_rating[final_rating['Num-Of-Rating']>=50]

In [43]:
final_rating.sample(10)

,User-ID,ISBN,Book-Rating,Title,Author,Year,Publisher,Image_url,Num-Of-Rating
131663,207499,0671027352,8,Angels &amp; Demons,Dan Brown,2000,Atria,http://images.amazon.com/images/P/0671027352.0...,193
12774,36836,0399135782,0,The Kitchen God's Wife,Amy Tan,1991,Putnam Pub Group,http://images.amazon.com/images/P/0399135782.0...,111
41439,26544,0515128546,0,Tears of the Moon (Irish Trilogy),Nora Roberts,2000,Jove Books,http://images.amazon.com/images/P/0515128546.0...,79
23416,164533,0156012197,9,The Little Prince,Antoine de Saint-ExupÃ©ry,2000,Harvest Books,http://images.amazon.com/images/P/0156012197.0...,53
14952,217375,0345443284,0,While I Was Gone,Sue Miller,1999,Ballantine Books,http://images.amazon.com/images/P/0345443284.0...,140
6105,245963,0425175405,0,Black Notice,Patricia Daniels Cornwell,2000,Berkley Publishing Group,http://images.amazon.com/images/P/0425175405.0...,123
2445,167471,0316776963,0,Me Talk Pretty One Day,David Sedaris,2001,Back Bay Books,http://images.amazon.com/images/P/0316776963.0...,146
183398,241980,0553578693,10,The Next Accident,LISA GARDNER,2002,Bantam,http://images.amazon.com/images/P/0553578693.0...,63
5907,245963,0399141464,0,Cause of Death,Patricia Daniels Cornwell,1996,Putnam Pub Group,http://images.amazon.com/images/P/0399141464.0...,121
69347,52199,0804115613,0,Fried Green Tomatoes at the Whistle Stop Cafe,Fannie Flagg,2000,Ballantine Books,http://images.amazon.com/images/P/0804115613.0...,131


In [44]:
final_rating.shape

(61853, 9)

In [45]:
final_rating.drop_duplicates(['User-ID', 'Title'], inplace=True)

In [46]:
final_rating.shape

(59850, 9)

In [47]:
final_rating.head()

,User-ID,ISBN,Book-Rating,Title,Author,Year,Publisher,Image_url,Num-Of-Rating
0,277427,002542730X,10,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,http://images.amazon.com/images/P/002542730X.0...,82
1,3363,002542730X,0,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,http://images.amazon.com/images/P/002542730X.0...,82
2,11676,002542730X,6,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,http://images.amazon.com/images/P/002542730X.0...,82
3,12538,002542730X,10,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,http://images.amazon.com/images/P/002542730X.0...,82
4,13552,002542730X,0,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,http://images.amazon.com/images/P/002542730X.0...,82


In [48]:
book_pivot = final_rating.pivot_table(columns= 'User-ID', index='Title', values='Book-Rating')

> In collaborative filtering recommendation systems, data is typically represented in a user-item matrix, where rows represent users, columns represent items (books in this case), and the cells represent the ratings given by users to items. However, this matrix is usually very sparse because not all users rate all items. A pivot table reshapes the data so that rows represent items, columns represent users, and the cells contain ratings. This makes it easier to work with the data.

In [49]:
book_pivot

User-ID,254,2276,2766,2977,3363,3757,4017,4385,6242,6251,...,274004,274061,274301,274308,274808,275970,277427,277478,277639,278418
Title,,,,,,,,,,,,,,,,,,,,,
1984,9.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN
1st to Die: A Novel,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2nd Chance,NaN,10.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,0.0,NaN
4 Blondes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
84 Charing Cross Road,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,10.0,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Year of Wonders,NaN,NaN,NaN,7.0,NaN,NaN,NaN,NaN,7.0,NaN,...,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN
You Belong To Me,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Zen and the Art of Motorcycle Maintenance: An Inquiry into Values,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,0.0,...,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN


In [50]:
book_pivot.fillna(0, inplace=True)

In [51]:
book_pivot

User-ID,254,2276,2766,2977,3363,3757,4017,4385,6242,6251,...,274004,274061,274301,274308,274808,275970,277427,277478,277639,278418
Title,,,,,,,,,,,,,,,,,,,,,
1984,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1st to Die: A Novel,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2nd Chance,0.0,10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4 Blondes,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
84 Charing Cross Road,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,10.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Year of Wonders,0.0,0.0,0.0,7.0,0.0,0.0,0.0,0.0,7.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
You Belong To Me,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Zen and the Art of Motorcycle Maintenance: An Inquiry into Values,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [52]:
from scipy.sparse import csr_matrix 

In [53]:
book_sparse = csr_matrix(book_pivot)

In [54]:
book_sparse

<742x888 sparse matrix of type '<class 'numpy.float64'>'
	with 14942 stored elements in Compressed Sparse Row format>

In [55]:
from sklearn.neighbors import NearestNeighbors
model = NearestNeighbors(algorithm = 'brute')

In [56]:
model.fit(book_sparse)

NearestNeighbors(algorithm='brute')

In [57]:
distance, suggestion = model.kneighbors(book_pivot.iloc[237,:].values.reshape(1, -1), n_neighbors=6)

In [58]:
distance

array([[ 0.        , 68.78953409, 69.5413546 , 72.64296249, 76.83098333,
        77.28518616]])

In [59]:
suggestion

array([[237, 240, 238, 241, 184, 536]], dtype=int64)

In [60]:
for i in range(len(suggestion)):
    print(book_pivot.index[suggestion[i]])

Index(['Harry Potter and the Chamber of Secrets (Book 2)',
       'Harry Potter and the Prisoner of Azkaban (Book 3)',
       'Harry Potter and the Goblet of Fire (Book 4)',
       'Harry Potter and the Sorcerer's Stone (Book 1)', 'Exclusive',
       'The Cradle Will Fall'],
      dtype='object', name='Title')


In [61]:
book_pivot.index

Index(['1984', '1st to Die: A Novel', '2nd Chance', '4 Blondes',
       '84 Charing Cross Road', 'A Bend in the Road', 'A Case of Need',
       'A Child Called \It\": One Child's Courage to Survive"',
       'A Civil Action', 'A Cry In The Night',
       ...
       'Winter Solstice', 'Wish You Well', 'Without Remorse',
       'Wizard and Glass (The Dark Tower, Book 4)', 'Wuthering Heights',
       'Year of Wonders', 'You Belong To Me',
       'Zen and the Art of Motorcycle Maintenance: An Inquiry into Values',
       'Zoya', '\O\" Is for Outlaw"'],
      dtype='object', name='Title', length=742)

In [62]:
book_name=book_pivot.index

In [63]:
import pickle 

In [64]:
pickle.dump(model, open('artifacts/NearestNeighbors/model.pkl', 'wb'))
pickle.dump(book_name, open('artifacts/NearestNeighbors/book_name.pkl', 'wb'))
pickle.dump(final_rating ,open('artifacts/NearestNeighbors/final_rating.pkl', 'wb'))
pickle.dump(book_pivot ,open('artifacts/NearestNeighbors/book_pivot.pkl', 'wb'))

In [65]:
def recommend_book_knn(book_name):
    book_id = np.where(book_pivot.index == book_name)[0][0]
    distance, suggestion = model.kneighbors(book_pivot.iloc[book_id,:].values.reshape(1, -1), n_neighbors=6)
    
    recommended_books = []
    for i in range(len(suggestion)):
        books = book_pivot.index[suggestion[i]]
        recommended_books.extend(books)

    return recommended_books
    
    #for i in range(len(suggestion)):
        #books = book_pivot.index[suggestion[i]]
        #for j in books:
            #print(j)


In [66]:
book_name = 'Harry Potter and the Chamber of Secrets (Book 2)'

In [67]:
recommend_book_knn(book_name)

['Harry Potter and the Chamber of Secrets (Book 2)',
 'Harry Potter and the Prisoner of Azkaban (Book 3)',
 'Harry Potter and the Goblet of Fire (Book 4)',
 "Harry Potter and the Sorcerer's Stone (Book 1)",
 'Exclusive',
 'The Cradle Will Fall']

### A different algorithm for the recommender system using matrix factorization with singular value decomposition (SVD) 

In [64]:
from sklearn.decomposition import TruncatedSVD

In [65]:
import pickle

In [66]:
svd = TruncatedSVD(n_components=50, random_state=42)
book_svd = svd.fit_transform(book_pivot)

In [67]:
book_name=book_pivot.index

In [68]:
pickle.dump(svd, open('artifacts/SVD/svd_model.pkl', 'wb'))
pickle.dump(book_name, open('artifacts/SVD/book_name.pkl', 'wb'))
pickle.dump(final_rating, open('artifacts/SVD/final_rating.pkl', 'wb'))
pickle.dump(book_svd, open('artifacts/SVD/book_svd.pkl', 'wb'))
pickle.dump(book_pivot ,open('artifacts/SVD/book_pivot.pkl', 'wb'))

In [69]:
def recommend_book_svd(book_name):
    book_id = np.where(book_pivot.index == book_name)[0][0]
    similarities = np.dot(book_svd, book_svd[book_id])
    closest = np.argsort(similarities)[-6:-1]
    recommended_books = [book_pivot.index[i] for i in closest]
    
    return recommended_books
    #for book in recommended_books:
        #print(book)

In [70]:
book_name = 'Harry Potter and the Chamber of Secrets (Book 2)'
recommend_book_svd(book_name)

['Harry Potter and the Order of the Phoenix (Book 5)',
 "Harry Potter and the Sorcerer's Stone (Book 1)",
 "Harry Potter and the Sorcerer's Stone (Harry Potter (Paperback))",
 'Harry Potter and the Goblet of Fire (Book 4)',
 'Harry Potter and the Prisoner of Azkaban (Book 3)']

### A hybrid approach using both collaborative filtering (matrix factorization with SVD) and content-based filtering (using book attributes)

In [73]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

In [74]:
#TF-IDF vectorizer for content-based filtering
tfidf_vectorizer = TfidfVectorizer(stop_words='english')
books['Author'] = books['Author'].fillna('')
books['Year'] = books['Year'].fillna('')
books['Publisher'] = books['Publisher'].fillna('')
books['features'] = books['Author'] + ' ' + books['Year'].astype(str) + ' ' + books['Publisher']
tfidf_matrix = tfidf_vectorizer.fit_transform(books['features'])

In [75]:
pickle.dump(svd, open('artifacts/hybrid/svd_model.pkl', 'wb'))
pickle.dump(book_pivot.index, open('artifacts/hybrid/book_name.pkl', 'wb'))
pickle.dump(final_rating, open('artifacts/hybrid/final_rating.pkl', 'wb'))
pickle.dump(book_svd, open('artifacts/hybrid/book_svd.pkl', 'wb'))
pickle.dump(tfidf_vectorizer, open('artifacts/hybrid/tfidf_vectorizer.pkl', 'wb'))
pickle.dump(tfidf_matrix, open('artifacts/hybrid/tfidf_matrix.pkl', 'wb'))

In [76]:
def hybrid_recommendation(book_name):
    # Collaborative filtering
    book_id = np.where(book_pivot.index == book_name)[0][0]
    similarities_svd = np.dot(book_svd, book_svd[book_id])
    closest_svd = np.argsort(similarities_svd)[-6:-1]
    recommended_books_svd = [book_pivot.index[i] for i in closest_svd]
    
    # Content-based filtering
    book_idx = np.where(books['Title'] == book_name)[0][0]
    cosine_similarities = linear_kernel(tfidf_matrix[book_idx], tfidf_matrix).flatten()
    related_books_idx = cosine_similarities.argsort()[-6:-1][::-1]
    recommended_books_content = [books['Title'][i] for i in related_books_idx]
    
    # Combine recommendations
    hybrid_recommendations = set(recommended_books_svd).union(set(recommended_books_content))
    return list(hybrid_recommendations)

In [77]:
book_name = 'Harry Potter and the Chamber of Secrets (Book 2)'
recommendations = hybrid_recommendation(book_name)
for book in recommendations:
    print(book)

Harry Potter and the Goblet of Fire (Book 4)
Harry Potter and the Prisoner of Azkaban (Book 3)
Harry Potter and the Sorcerer's Stone (Harry Potter (Paperback))
Harry Potter and the Chamber of Secrets Postcard Book
Harry Potter and the Order of the Phoenix (Book 5)
Harry Potter and the Sorcerer's Stone (Book 1)
Harry Potter and the Chamber of Secrets (Book 2)
